In [4]:
import lightgbm
import pandas as pd
import numpy as np

In [2]:
init_record = pd.read_csv("../../Data/LWB_Monthly_data.csv")


In [3]:
init_record['_key_occurreddate_month']=init_record['_key_occurreddate_month'].astype("datetime64[ns]")
init_record = init_record.set_index(['_key_client_id', '_key_occurreddate_month'])

In [6]:
def construct_feature_label(data):
    y = []
    clientList = list(set(data.index.get_level_values(0)))
    data['response_variable'] = data.groupby(level=0)['response_variable'].shift(-1)  
    data = data.groupby(level=0, as_index=False).apply(lambda group: group.iloc[:-1])
    #print(data.loc["0000D172-EA88-432F-8235-9FAA00D29072"]['response_variable'])
    y = data['response_variable']
    x = data.drop('response_variable',axis='columns')
    return x, y

feature, label = construct_feature_label(init_record)


# PCA

In [7]:
from sklearn import decomposition


In [16]:
pca = decomposition.PCA(n_components=0.9)
feature_pca = pca.fit_transform(feature)

# Model

In [22]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score, confusion_matrix, roc_curve


In [18]:
model=IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=1.0)
model.fit(feature_pca)

IsolationForest(contamination=0.1, n_estimators=50)

In [19]:
y_hat = model.predict(feature_pca)

In [20]:
y_hat = np.where(y_hat == -1, 1, 0) 

In [23]:
f1_score(label, y_hat)

0.08793157400646709